In [1]:
import wandb
import torch
import pandas as pd
from grelu.lightning import LightningModel
import pytorch_lightning as pl
from grelu.sequence.utils import get_unique_length, resize

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## set up wandb

In [2]:
wandb.login(host="https://api.wandb.ai", relogin=True)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: avantikalal (grelu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
run = wandb.init(entity='grelu', project='borzoi', job_type='copy', name='copy-mouse') # Initialize a W&B Run

## Paths

In [4]:
tasks_path = '/code/borzoi/data/targets_mouse.txt.gz'
intervals_path = '/gstore/data/resbioai/grelu/borzoi-data/mm10/sequences.bed'

## Process tasks

In [5]:
tasks = pd.read_table(tasks_path, index_col=0)
print(len(tasks))

2608


In [6]:
tasks=tasks.reset_index(drop=True)
tasks["assay"] = tasks["description"].apply(lambda x: x.split(":")[0])
tasks["assay"].value_counts()
tasks["sample"] = tasks["description"].apply(lambda x: ":".join(x.split(":")[1:]))
tasks = tasks.rename(columns={"identifier":"name"})
tasks.head(2)

,name,file,clip,clip_soft,scale,sum_stat,strand_pair,description,assay,sample
0,CNhs10464+,/home/drk/tillage/datasets/mouse/cage/fantom/C...,768,384,1.0,sum,1,"CAGE:placenta, adult pregnant day17",CAGE,"placenta, adult pregnant day17"
1,CNhs10464-,/home/drk/tillage/datasets/mouse/cage/fantom/C...,768,384,1.0,sum,0,"CAGE:placenta, adult pregnant day17",CAGE,"placenta, adult pregnant day17"


In [7]:
tasks = tasks.to_dict(orient="list")

## Process intervals

In [8]:
intervals = pd.read_table(intervals_path, header=None)
intervals.columns = ['chrom', 'start', 'end', 'fold']
intervals.head()

,chrom,start,end,fold
0,chr1,46257174,46453782,fold0
1,chr2,83512641,83709249,fold0
2,chr7,16218353,16414961,fold0
3,chr3,113724419,113921027,fold0
4,chr3,107470140,107666748,fold0


In [9]:
intervals['split'] = 'train'
intervals.loc[intervals.fold=='fold3', 'split'] = 'test'
intervals.loc[intervals.fold=='fold4', 'split'] = 'val'
intervals.split.value_counts()

split
train    36950
val       6318
test      6101
Name: count, dtype: int64

In [10]:
get_unique_length(intervals)

196608

In [11]:
intervals = resize(intervals, 524288)

In [12]:
train_intervals = intervals[intervals.split=='train'].iloc[:, :3]
val_intervals = intervals[intervals.split=='val'].iloc[:, :3]
test_intervals = intervals[intervals.split=='test'].iloc[:, :3]
del intervals

## Initialize model

In [13]:
model_params = {
    "model_type":"BorzoiModel",
    "n_tasks":2608,
    "final_act_func": 'softplus',
    "final_pool_func":None,
    "crop_len":5120,
}
train_params={
    "task":"regression",
    "loss":"mse",
}
lm = LightningModel(model_params, train_params)

## Save checkpoints

In [14]:
from grelu.sequence.format import convert_input_type
input = convert_input_type(['A'*524288], "one_hot")
input.shape

torch.Size([1, 4, 524288])

In [15]:
6144*32

196608

In [17]:
for rep in range(4):
    state_dict = torch.load(f"/data/borzoi/torch_weights/mouse_fold{rep}.h5")
    lm.model.load_state_dict(state_dict)
    lm.data_params["tasks"] = tasks

    lm.data_params["train"] = dict()
    lm.data_params["val"] = dict()
    lm.data_params["test"] = dict()
    
    lm.data_params["train"]["seq_len"] = 524288
    lm.data_params["train"]["label_len"] = 6144*32
    lm.data_params["train"]["genome"] = "hg38"
    lm.data_params["train"]["bin_size"] = 32
    
    lm.data_params["train"]["intervals"] = train_intervals.to_dict(orient='list')
    lm.data_params["val"]["intervals"] = val_intervals.to_dict(orient='list')
    lm.data_params["test"]["intervals"] = test_intervals.to_dict(orient='list')
    
    assert lm(input).shape == (1, 2608, 6144)
    
    trainer = pl.Trainer()
    try:
        trainer.predict(lm)
    except:
        pass
    
    trainer.save_checkpoint(f'/data/borzoi/torch_weights/mouse_rep{rep}.ckpt')

/tmp/ipykernel_3287790/3322024732.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f"/data/borzoi/torch_weights/mouse_fold{rep}.h5")
Trainer will 

## Upload to wandb

In [18]:
metadata={
            'model_params':lm.model_params, 
            'train_params':lm.train_params, 
            'data_params':lm.data_params
        }

In [19]:
for rep in range(4):
    ckpt = f'/data/borzoi/torch_weights/mouse_rep{rep}.ckpt'
    artifact = wandb.Artifact(
        f'mouse_rep{rep}', 
        type='model',
        metadata=metadata,
    )
    artifact.add_file(ckpt)
    run.log_artifact(artifact)

wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list that is 312024 bytes
wandb: WARNING Serializing object of type list t

In [20]:
run.finish()